# veFVM Distribution

In [9]:
import pandas as pd
import json

vefvm_total = 60000

In [2]:
bsc_mpx_balances = pd.read_csv("../products/bsc-mpx-balances.csv", header=0, names=["address", "bsc_mpx_balance"], dtype={"address": str, "bsc_mpx_balance": float})
bsc_thena_balances = pd.read_csv("../products/bsc-thena-balances.csv", header=0, names=["address", "bsc_thena_lp_balance", "bsc_thena_mpx_balance"], dtype={"address": str, "bsc_thena_lp_balance": float, "bsc_thena_mpx_balance": float})
fantom_equalizer_balances = pd.read_csv("../products/fantom-equalizer-balances.csv", header=0, names=["address", "fantom_equalizer_lp_balance", "fantom_equalizer_mpx_balance"], dtype={"address": str, "fantom_equalizer_lp_balance": float, "fantom_equalizer_mpx_balance": float})
fantom_fsmlp_balances = pd.read_csv("../products/fantom-fsmlp-balances.csv", header=0, names=["address", "fantom_fsmlp_balance"], dtype={"address": str, "fantom_fsmlp_balance": float})
fantom_fvm_balances = pd.read_csv("../products/fantom-fvm-balances.csv", header=0, names=["address", "fantom_fvm_lp_balance", "fantom_fvm_mpx_balance"], dtype={"address": str, "fantom_fvm_lp_balance": float, "fantom_fvm_mpx_balance": float})
fantom_mpx_balances = pd.read_csv("../products/fantom-mpx-balances.csv", header=0, names=["address", "fantom_mpx_balance"], dtype={"address": str, "fantom_mpx_balance": float})
fantom_smpx_balances = pd.read_csv("../products/fantom-smpx-balances.csv", header=0, names=["address", "fantom_smpx_balance"], dtype={"address": str, "fantom_smpx_balance": float})

In [3]:
bsc_combined = pd.merge(bsc_mpx_balances, bsc_thena_balances, on=["address"], how="outer")
bsc_combined.fillna(0, inplace=True)
len(bsc_combined)
bsc_combined.head()

,address,bsc_mpx_balance,bsc_thena_lp_balance,bsc_thena_mpx_balance
0,0x3461e03e15329f64B95264e4FFD7079281ae1e48,8.910629e+18,8.879953e+19,3.710819e+21
1,0xcC28906983a61541632fb9B8ae2e1bEe531C4aaA,2.765961e+17,0.000000e+00,0.000000e+00
2,0xc01895BF22E746Af53A6d820a3224521CEe0B916,6.531759e+15,3.523060e+19,1.472242e+21
3,0x3E7b8D12D4De6CB284A7d60Bf92B462D2a6d6A2F,8.720234e+15,0.000000e+00,0.000000e+00
4,0xD7FF20a8c5Ac84cf1aea70908F5d3CC36951d121,1.640000e+12,0.000000e+00,0.000000e+00


In [4]:
ftm_combined = pd.merge(fantom_equalizer_balances, fantom_fsmlp_balances, on=["address"], how="outer")
ftm_combined = pd.merge(ftm_combined, fantom_fvm_balances, on=["address"], how="outer")
ftm_combined = pd.merge(ftm_combined, fantom_mpx_balances, on=["address"], how="outer")
ftm_combined = pd.merge(ftm_combined, fantom_smpx_balances, on=["address"], how="outer")
ftm_combined.fillna(0, inplace=True)
ftm_combined.head()
len(ftm_combined)

2528

In [5]:
combined = pd.merge(bsc_combined, ftm_combined, on=["address"], how="outer")
combined.fillna(0, inplace=True)
len(combined)

2560

In [6]:
combined["total_mpx"] = \
    combined["bsc_thena_mpx_balance"] + \
    combined["fantom_equalizer_mpx_balance"] + \
    combined["fantom_fvm_mpx_balance"] + \
    combined["fantom_smpx_balance"] + \
    combined["bsc_mpx_balance"] + \
    combined["fantom_mpx_balance"]

In [11]:
# Remove treasury, mpx, etc
with open("../data/excluded.json", "r") as f:
    excluded = json.load(f)
combined = (combined[~combined.address.isin(excluded)])

In [12]:
mpx_total = sum(combined["total_mpx"])
combined['vefvm_balance'] = (combined['total_mpx'] / mpx_total) * vefvm_total
sum(combined['vefvm_balance'])

59999.9999999999

In [13]:
combined.to_csv("../products/combined.csv", index=False, float_format='%.0f')

In [14]:
vefvm = combined[["address", "vefvm_balance"]]
vefvm.to_csv("../products/vefvm-balances.csv", index=False, float_format='%.0f')